In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1



##### Attemp1: Dropping two non-beneficial columns 'STATUS' and 'SPECIAL_CONSIDERATIONS' to improve accuracy


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],axis=1)

In [4]:
counts_df = application_df['APPLICATION_TYPE'].value_counts()
counts_df

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
application_types_to_replace = list(counts_df[counts_df<400].index)
#application_types_to_replace
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
countsbinning = application_df['CLASSIFICATION'].value_counts()
countsclassification = countsbinning[countsbinning>1]
#countsclassification
classifications_to_replace = list (countsbinning[countsbinning<100].index)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

application_df = pd.get_dummies(application_df,dtype=float)
#application_df.head()

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
#y
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
#X
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

### Scaling and Fitting Data 

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


##### Attempt 1

In [7]:

number_input_features = len( X_train_scaled[0])
layer1=50
layer2=30

nn = tf.keras.models.Sequential()
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=number_input_features, activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=10)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2500      
                                                                 
 dense_1 (Dense)             (None, 30)                1530      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,061
Trainable params: 4,061
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
684/684 [==============================] - 1s 1ms/step - loss: 0.5766 - accuracy: 0.7135 - val_loss: 0.5476 - val_accuracy: 0.7336
Epoch 2/10
684/684 [==============================] - 1s 907us/step - loss: 0.5553 - accuracy: 0.7260 - val_loss: 0.5450 - val_accuracy: 0.7362
Epoch 3/10
684/684 [==========

##### Attempt 2

In [8]:
number_input_features = len( X_train_scaled[0])
layer1=80
layer2=40

nn = tf.keras.models.Sequential()
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=number_input_features, activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.20, epochs=40)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                4000      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 7,281
Trainable params: 7,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
644/644 [==============================] - 1s 1ms/step - loss: 0.5748 - accuracy: 0.7179 - val_loss: 0.5543 - val_accuracy: 0.7351
Epoch 2/40
644/644 [==============================] - 1s 1ms/step - loss: 0.5583 - accuracy: 0.7243 - val_loss: 0.5477 - val_accuracy: 0.7359
Epoch 3/40
644/644 [============

##### Attempt 3

In [9]:
number_input_features = len( X_train_scaled[0])
layer1=100
layer2=100
layer3=50

nn = tf.keras.models.Sequential()
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=number_input_features, activation='tanh'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation='tanh'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=layer3, activation='tanh'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.30, epochs=100)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               5000      
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 20,201
Trainable params: 20,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
563/563 [==============================] - 2s 2ms/step - loss: 0.5763 - accuracy: 0.7129 - val_loss: 0.5653 - val_accuracy: 0.7102
Epoch 2/100
563/563 [==================